This notebook uses iFeatureOmega, a feature generation software, to add to our feature space for a RandomForestClassifier that predicts protein pair functionality.

To do:

1) Write unit tests for iFeatureOmega
2) time trial for different descriptors - DONE
3) Figure out how to append meso and thermo descriptors
4) Make sure protein length is in training
5) try changing subject_align_len to subject_align_len/m_protein_len
6) log(ratio) for proteins approacing zero or infinity
    take distributions to assess for this.
    Update -- taking log and then droping infinites and NaN improves score to 0.86 <-- cannot repeat this

In [44]:
import duckdb

con = duckdb.connect(path_to_db)
cmd = """SELECT list_of_column_names FROM fafsa_final"""
df = con.execute(cmd).df()

NameError: name 'path_to_db' is not defined

In [ ]:
output_df.to_parquet(path)
cmd = f"""CREATE OR REPLACE TABLE table_name AS SELECT * FROM '{path}'"""
con.execute(cmd)

 protein_pair_cmd = """CREATE OR REPLACE TABLE fafsa_protein_pairs AS
                          SELECT 
                          meso_pid,
                          thermo_pid,
                          bit_score,
                          local_gap_compressed_percent_id,
                          scaled_local_query_percent_id,
                          scaled_local_symmetric_percent_id,
                          query_align_len,
                          query_align_cov,
                          subject_align_len,
                          subject_align_cov,
                          FROM protein_pairs USING SAMPLE 100
"""

In [55]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_selection
import unittest
import iFeatureOmegaCLI
import Bio.SeqIO
import Bio.SeqRecord
import io
from io import StringIO
import time

In [56]:
def get_fasta_from_dataframe(dataframe, output_file_a, output_file_b):
    #adjust this to write function with BioPython
    #separate functions for each of the input sequences
    #in training, seq_a = meso and seq_b = thermo
    
    
    #meso sequence to fasta
    with open(output_file_a, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['prot_pair_index']), row['m_protein_seq']))
    
    #thermo sequence to fasta
    with open(output_file_b, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['prot_pair_index']), (row['t_protein_seq'])))
   
    #return output files
    return [output_file_a, output_file_b]

In [57]:
def get_protein_descriptors(fasta_file, descriptors=[]):
    
    """
    Generates features from a protein sequence

    Parameters
    ----------
    Fasta file with amino acid sequences.

    Returns
    -------
    Vector of descriptors
    """
    
    #create iProtein object
    protein = iFeatureOmegaCLI.iProtein(fasta_file)
    
    #not sure why we need this yet. Right now it is stored in local directory.
    params = protein.import_parameters('protein_parameters.json')
    
    protein_descriptors = {}
    
    for descriptor in descriptors:
        protein.get_descriptor(descriptor)
        protein_descriptors.update({f'{descriptor}':protein.encodings})
        
    return protein_descriptors

In [86]:
def create_new_dataframe(dataframe, output_files, descriptors=[]):
    """
    Creates new dataframe with descriptors added.

    Parameters
    ----------
    Pandas dataframe, list of descriptors as strings, output file name.

    Returns
    -------
    Dataframe including vector(s) of descriptors
    """

    fasta_files = get_fasta_from_dataframe(dataframe, output_files[0], output_files[1])
    
    def compute_descriptor_ratio(fasta_files, descriptors=[]):
        """
        Generates dictionary of descriptors for each of the two input sequences.
        Computes the difference between each instance of a descriptor.
        
        Parameters
        ----------
        List of two fasta files (str) and list of descriptors (str).

        Returns
        -------
        Dictionary with difference between descriptors for each of the 
        input sequences.
        """
        desc_a = get_protein_descriptors(fasta_files[0], descriptors)
        desc_b = get_protein_descriptors(fasta_files[1], descriptors)

        feature_dict = {}

        for key in desc_a:
            
            if 'AAC' in key:
                feature_dict[key] = desc_a[key] - desc_b[key]
            elif 'GAAC' in key:
                feature_dict[key] = desc_a[key] - desc_b[key]
            else:
                feature_dict[key] = desc_a[key] / desc_b[key]

        return feature_dict
    
    feature_dict = compute_descriptor_ratio(fasta_files, descriptors)


    df = dataframe.reset_index()

    for desc in descriptors:

        feature_dict[desc].index = feature_dict[desc].index.astype(int)
        features = feature_dict[desc].reset_index()

        df = pd.merge(
            df,
            features,
            how='outer',
            left_index=True,
            right_index=True)

    return df

In [59]:
#iFeature properties

# protein = iFeatureOmegaCLI.iProtein('meso_50k.fasta')
# protein.display_feature_types()

In [60]:
cd /Users/loganroberts/Learn2Therm/ValidProt/FAFSA

/Users/loganroberts/Learn2Therm/ValidProt/FAFSA


In [61]:
#convert to pandas df
df = pd.read_csv('learn2therm_sample_50k.csv')
df.columns

Index(['Unnamed: 0', 'local_gap_compressed_percent_id',
       'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id',
       'query_align_len', 'query_align_cov', 'subject_align_len',
       'subject_align_cov', 'bit_score', 'thermo_index', 'meso_index',
       'prot_pair_index', 'meso_protein_int_index', 'thermo_protein_int_index',
       'taxa_pair_index', 'local_gap_compressed_percent_id_16s',
       'scaled_local_query_percent_id_16s',
       'scaled_local_symmetric_percent_id_16s', 'query_align_cov_16s',
       'subject_align_cov_16s', 'bit_score_16s', 'm_ogt', 't_ogt',
       'ogt_difference', 'm_protein_seq', 't_protein_seq', 'm_protein_desc',
       't_protein_desc', 'm_protein_len', 't_protein_len'],
      dtype='object')

In [62]:
cd /Users/loganroberts/Learn2Therm/ValidProt/notebooks

/Users/loganroberts/Learn2Therm/ValidProt/notebooks


In [63]:
target = pd.read_csv('protein_match_6k.csv')

In [64]:
df

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,875,...,1153.0,27.5,50.0,22.5,MAESGTSRRADHLVPVPGPDAEPPAVADELLRAVGRGDEQAFGRLY...,MPSQITESERIELAERFERDALPLLDQLYSAALRMTRNPADAEDLV...,ECF RNA polymerase sigma factor SigK,sigma-70 family RNA polymerase sigma factor,206,202
1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,11324,...,1014.0,25.0,54.0,29.0,MARIALVDDDRNILTSVSMTLEAEGFEVETYNDGQSALDAFNKRMP...,MRVLLVEDDPNTSRSIEMMLTHANLNVYATDMGEEGIDLAKLYDYD...,response regulator transcription factor,response regulator transcription factor,233,237
2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,875,...,1138.0,28.0,50.0,22.0,MKDTVVFVTGAARGIGAHTARLAVARGARVALVGLEPHLLADLAAE...,MTPEQIFSGQTAIVTGGASGIGAATVEHIARRGGRVFSVDLSYDSP...,SDR family oxidoreductase,SDR family oxidoreductase,287,252
3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,875,...,1077.0,28.0,50.0,22.0,MTSGLWERVLDGVWVTIQLLVLSALLATAVSFVVGIARTHRLWIVR...,MAMSRRKRGQLARGIQYAILVIVVVVLALLADWGKIGKAFFDWEAA...,ectoine/hydroxyectoine ABC transporter permeas...,amino acid ABC transporter permease,234,269
4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,9827,...,991.0,30.0,50.0,20.0,MIISLRRGLRFIRFIVFFAALVYLFYHVLDLFNGWISPVDQYQMPT...,MKRMVWRTLKVFIIFIACTLLFYFGLRFMHLEYEQFHRYEPPEGPA...,YqzK family protein,YqzK family protein,80,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0.344828,0.169972,0.192308,180,0.509915,200,0.738007,53,14963,...,1055.0,28.0,52.5,24.5,MDNATFRLGDDLSVRLPGHSRWIGQVEREQRWLPWLAPRLPLTVST...,MPPQPRPLRPNDPREIGGFALLGRLGEGGQGTVYLGGAPDGRRVAV...,aminoglycoside phosphotransferase family protein,serine/threonine protein kinase,271,353
49996,49996,0.417989,0.389163,0.392060,190,0.935961,187,0.935000,314,7134,...,1083.0,26.5,52.5,26.0,MFRTGVKAEIGRSLAVVGEAEDVERAVRVVLEQRPDVVLLDVHLPG...,MILEAEPDIVVVGEAGDGEKAVEEARALQPDVVLMDIRMPRKDGVE...,response regulator transcription factor,response regulator transcription factor,200,203
49997,49997,0.352174,0.239645,0.242152,230,0.680473,227,0.685801,279,7134,...,991.0,28.0,52.5,24.5,MIRLAELTKTYPGQQHPAVDGISMEVAEGEIVVLVGPSGCGKTTTL...,MTEQPILSARGLTVDFRLRGGRRARAVDGVDLDLAPGEVLALAGES...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,331,338
49998,49998,0.334764,0.331915,0.329810,232,0.987234,230,0.966387,281,11324,...,907.0,29.0,54.0,25.0,MSESHAGALLSVRGLTAGYGGATALDGVSLTVAAGETVALLGANGA...,MSLLTTSGLTRHFSGIHAVEGVDFTLEAGEIRALIGSNGAGKTTLV...,ABC transporter ATP-binding protein,ABC transporter ATP-binding protein,238,235


In [65]:
from sklearn.utils import resample

# Assuming your data is in a pandas DataFrame called 'data'
# Separate the majority and minority classes
majority_class = target[target['protein_match'] == 'Yes']
minority_class = target[target['protein_match'] == 'No']

# Undersample the majority class to match the number of minority class samples
n_samples = len(minority_class)
undersampled_majority = resample(majority_class, n_samples=n_samples, replace=False)

# Combine the undersampled majority class with the minority class
balanced_data = pd.concat([undersampled_majority, minority_class])

In [66]:
balanced_data

,prot_pair_index,Jaccard_Score,protein_match
3566,135383672,0.750000,Yes
416,163772252,0.500000,Yes
6156,66697787,0.400000,Yes
1332,153454786,1.000000,Yes
5167,138989653,0.666667,Yes
...,...,...,...
6328,125335561,0.250000,No
6329,105441342,0.200000,No
6336,87888523,0.200000,No
6339,38582871,0.200000,No


In [67]:
df = pd.merge(df, balanced_data, on=['prot_pair_index'])
df.shape

(2210, 32)

In [68]:
"""
this list comes from a combination of reading through the features and determining which might be useful
and timing some of the feature generations. those that took more than 30ish seconds were eliminated
Also removed those that have really high dimensionality (>4000)
"""

feature_list = ['AAC', 'GAAC', 'DistancePair',
               'CTDC', 'CTDT', 'CTDD', 'CTriad', 'GDPC type 1', 'GDPC type 2',
                'CKSAAGP type 1', 'CKSAAGP type 2', 'PseKRAAC type 2', 'PseKRAAC type 3A','PseKRAAC type 7',
                'PseKRAAC type 9', 'Geary','APAAC', 'QSOrder']

In [69]:
# df = create_new_dataframe(df, 'seq_50k.fasta', descriptors= ['EGAAC', 'CKSAAGP type 1', 'CKSAAGP type 2',
#                                                             'GDPC type 1', 'GDPC type 2', 'TPC type 1', 'TPC type 2',
#                                                             'GTPC type 1', 'GTPC type 2', 'DPC type 1', 'DPC type 2',
#                                                             'Moran', 'Geary', 'NMBroto', 'AC', 'CC', 'ACC',
#                                                             'PAAC', 'APAAC'])

In [70]:
df = create_new_dataframe(df, ['seq_50k_a.fasta', 'seq_50k_b.fasta'], descriptors=['AAC', 'GAAC'])

File imported successfully.
File imported successfully.


Split data into dev and test, and then split that into train and validation.

In [71]:
#drop columns that don't exihibit signficant pearson correlation with bit_score

df = df.drop(columns = ['meso_index', 'meso_protein_int_index', 'local_gap_compressed_percent_id_16s', 
                        'scaled_local_query_percent_id_16s', 'scaled_local_symmetric_percent_id_16s',
                       'bit_score_16s', 'm_ogt', 't_ogt', 'taxa_pair_index', 'thermo_protein_int_index'
                       , 'prot_pair_index', 'ogt_difference', 'Jaccard_Score',
                       'query_align_cov_16s', 'subject_align_cov_16s',
                       'Unnamed: 0', 'thermo_index','m_protein_seq', 't_protein_seq', 
                       'm_protein_desc', 't_protein_desc'])

In [72]:
df=df.drop(columns=['index_y', 'index_x'])
df

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,m_protein_len,t_protein_len,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0.333333,0.267380,0.254453,148,0.791444,163,0.791262,127,206,187,...,1.008630,1.361650,1.815534,0.907767,20752996,1.022542,0.847249,0.972607,1.105108,0.961165
1,0.268482,0.190083,0.189300,248,0.683196,271,0.740437,132,366,363,...,1.172131,1.024863,0.991803,0.826503,86923756,1.181320,0.771403,0.797755,0.940942,0.876253
2,0.265306,0.201550,0.210243,320,0.826873,294,0.828169,140,355,387,...,1.392958,0.895473,inf,0.631134,180139301,1.070789,0.654085,0.733749,0.838570,1.283330
3,0.275281,0.182156,0.152174,185,0.687732,197,0.525333,48,375,269,...,0.797037,0.947905,1.255333,0.358667,37128170,1.145322,0.868351,1.021657,1.099911,0.727882
4,0.394495,0.346774,0.369099,108,0.870968,106,0.972477,130,109,124,...,1.422018,0.796330,0.000000,NaN,166997534,0.945095,0.758410,0.812582,0.948012,1.327217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0.439614,0.393939,0.383158,210,0.909091,212,0.868852,400,244,231,...,1.798770,0.899385,0.946721,0.946721,161535261,0.849622,1.622951,1.007800,1.012012,1.269009
2206,0.241758,0.134557,0.144975,181,0.553517,190,0.678571,107,280,327,...,0.568956,0.904147,1.459821,0.917602,17777778,1.111864,1.534898,0.755672,1.094866,0.705580
2207,0.321429,0.186391,0.223801,197,0.582840,192,0.853333,195,225,338,...,1.386667,1.024242,3.004444,0.563333,87888523,1.195430,0.795294,0.926903,0.782407,0.876296
2208,0.290909,0.162437,0.154589,110,0.558376,112,0.516129,85,217,197,...,1.513057,1.664363,0.907834,1.210445,38582871,0.866190,1.513057,0.975081,1.307281,1.149923


In [46]:
df.columns

Index(['local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'query_align_len',
       'query_align_cov', 'subject_align_len', 'subject_align_cov',
       'bit_score', 'm_protein_len', 't_protein_len', 'protein_match', 'AAC_A',
       'AAC_C', 'AAC_D', 'AAC_E', 'AAC_F', 'AAC_G', 'AAC_H', 'AAC_I', 'AAC_K',
       'AAC_L', 'AAC_M', 'AAC_N', 'AAC_P', 'AAC_Q', 'AAC_R', 'AAC_S', 'AAC_T',
       'AAC_V', 'AAC_W', 'AAC_Y', 'index', 'GAAC_alphatic', 'GAAC_aromatic',
       'GAAC_postivecharge', 'GAAC_negativecharge', 'GAAC_uncharge'],
      dtype='object')

In [73]:
df=df.drop(columns=['local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'query_align_len',
       'query_align_cov', 'subject_align_len', 'subject_align_cov',
       'bit_score', 'm_protein_len', 't_protein_len'])

In [74]:
df

,protein_match,AAC_A,AAC_C,AAC_D,AAC_E,AAC_F,AAC_G,AAC_H,AAC_I,AAC_K,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,Yes,0.839684,inf,1.567961,0.680825,0.635437,1.180097,0.680825,3.177184,inf,...,1.008630,1.361650,1.815534,0.907767,20752996,1.022542,0.847249,0.972607,1.105108,0.961165
1,Yes,1.405055,inf,1.416862,0.385701,0.721311,1.395871,0.991803,1.166827,1.190164,...,1.172131,1.024863,0.991803,0.826503,86923756,1.181320,0.771403,0.797755,0.940942,0.876253
2,Yes,1.348332,1.816901,0.961889,0.778672,0.613204,0.965553,0.218028,1.557344,0.915718,...,1.392958,0.895473,inf,0.631134,180139301,1.070789,0.654085,0.733749,0.838570,1.283330
3,No,1.716476,0.430400,1.434667,0.843922,1.229714,0.938051,0.807000,0.896667,0.239111,...,0.797037,0.947905,1.255333,0.358667,37128170,1.145322,0.868351,1.021657,1.099911,0.727882
4,Yes,0.616208,1.820183,1.706422,0.731324,1.137615,1.023853,2.275229,1.706422,NaN,...,1.422018,0.796330,0.000000,NaN,166997534,0.945095,0.758410,0.812582,0.948012,1.327217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,Yes,0.680456,0.946721,1.549180,0.683743,2.130123,0.817623,0.788934,1.104508,5.206967,...,1.798770,0.899385,0.946721,0.946721,161535261,0.849622,1.622951,1.007800,1.012012,1.269009
2206,Yes,0.824370,2.335714,0.875893,1.313839,2.245879,1.475188,0.583929,1.648739,2.335714,...,0.568956,0.904147,1.459821,0.917602,17777778,1.111864,1.534898,0.755672,1.094866,0.705580
2207,No,1.220556,0.429206,1.126667,0.536508,0.751111,1.306280,0.938889,0.643810,0.858413,...,1.386667,1.024242,3.004444,0.563333,87888523,1.195430,0.795294,0.926903,0.782407,0.876296
2208,No,0.556414,inf,1.361751,1.257001,2.269585,0.756528,0.181567,0.453917,2.118280,...,1.513057,1.664363,0.907834,1.210445,38582871,0.866190,1.513057,0.975081,1.307281,1.149923


In [19]:
#maybe divide all of the AAC/GAAC columns by the protein length 

In [94]:
df.shape

(2210, 72)

In [75]:
df = df.replace([np.inf, -np.inf], np.nan)

In [78]:
nan_counts = df.isna().sum()
print(nan_counts)
nan_counts.unique()

protein_match          0
AAC_A                  0
AAC_D                  0
AAC_G                  0
AAC_L                  0
AAC_S                  0
AAC_V                  0
index                  0
GAAC_alphatic          0
GAAC_postivecharge     0
GAAC_negativecharge    0
GAAC_uncharge          0
dtype: int64


array([0])

In [107]:
df.columns[36:]

Index(['QSOrder_Grantham.Xd.2', 'QSOrder_Grantham.Xd.3'], dtype='object')

In [110]:
log_cols = df.columns[36:]
df[log_cols] = df[log_cols].apply(lambda x: np.log(x))

In [77]:
df = df.dropna(axis=1, how='any')

In [51]:
nan_counts = df.isna().sum()
nan_counts

protein_match            0
AAC_A                    0
AAC_C                  434
AAC_D                    3
AAC_E                    3
AAC_F                   20
AAC_G                    0
AAC_H                   31
AAC_I                    4
AAC_K                   58
AAC_L                    1
AAC_M                    1
AAC_N                   49
AAC_P                    7
AAC_Q                   20
AAC_R                    1
AAC_S                    0
AAC_T                    2
AAC_V                    0
AAC_W                  392
AAC_Y                   48
index                    0
GAAC_alphatic            0
GAAC_aromatic            2
GAAC_postivecharge       0
GAAC_negativecharge      0
GAAC_uncharge            0
dtype: int64

In [25]:
df.shape

(2210, 56)

Use MRMR to select for the best features. Going to start by grouping into different categories of features generated from iFeature Omega.

In [26]:
#use MRMR to select for the best features from PseKRAAC
df_subset = df.loc[:, df.columns != 'protein_match']
print(type(df_subset))

# select top 10 features using mRMR
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=df_subset.iloc[:,10:], y=df['protein_match'], K=20)

selected_features

<class 'pandas.core.frame.DataFrame'>


100%|██████████| 20/20 [00:00<00:00, 48.64it/s]


['QSOrder_Schneider.Xr.A',
 'QSOrder_Grantham.Xd.3',
 'QSOrder_Grantham.Xr.R',
 'QSOrder_Schneider.Xd.1',
 'AAC_A',
 'QSOrder_Schneider.Xd.3',
 'QSOrder_Grantham.Xr.A',
 'AAC_I',
 'QSOrder_Schneider.Xr.R',
 'AAC_S',
 'GAAC_alphatic',
 'AAC_E',
 'QSOrder_Grantham.Xd.2',
 'AAC_N',
 'GAAC_negativecharge',
 'QSOrder_Schneider.Xr.L',
 'QSOrder_Schneider.Xr.G',
 'AAC_T',
 'GAAC_uncharge',
 'AAC_R']

In [34]:
best_features_df = df[[feature for feature in selected_features]]

#concatenates original feature vector back into the dataframe
df = pd.concat([df.iloc[:, :11], best_features_df], axis=1)
df

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,m_protein_len,t_protein_len,...,GAAC_alphatic,AAC_E,QSOrder_Grantham.Xd.2,AAC_N,GAAC_negativecharge,QSOrder_Schneider.Xr.L,QSOrder_Schneider.Xr.G,AAC_T,GAAC_uncharge,AAC_R
0,0.283582,0.231707,0.240506,133,0.810976,134,0.881579,99,152,164,...,-0.015083,-0.013479,1.036564,0.006579,0.004974,1.329084,0.701461,0.003851,0.059050,-0.034981
1,0.268482,0.190083,0.189300,248,0.683196,271,0.740437,132,366,363,...,0.078422,-0.030461,0.982711,0.002394,-0.006345,0.920360,1.465759,0.010432,-0.035113,-0.030800
2,0.265306,0.201550,0.210243,320,0.826873,294,0.828169,140,355,387,...,0.030913,-0.020017,0.978314,0.011479,-0.021691,0.886583,1.065708,0.018277,0.057837,-0.010125
3,0.275281,0.182156,0.152174,185,0.687732,197,0.525333,48,375,269,...,0.064287,-0.009864,1.003861,0.005898,0.011143,1.139824,0.931587,-0.013581,-0.068788,0.016882
4,0.394495,0.346774,0.369099,108,0.870968,106,0.972477,130,109,124,...,-0.028781,-0.030334,0.917208,0.000000,-0.007547,1.961181,1.123222,0.013613,0.063332,-0.031444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0.241758,0.134557,0.144975,181,0.553517,190,0.678571,107,280,327,...,0.049945,0.007678,1.001902,0.024509,0.004642,0.949185,1.416965,-0.051409,-0.086435,-0.043283
2206,0.321429,0.186391,0.223801,197,0.582840,192,0.853333,195,225,338,...,0.082104,-0.038396,0.931608,0.002998,-0.030901,1.624165,1.373086,0.014872,-0.030743,-0.005786
2207,0.290909,0.162437,0.154589,110,0.558376,112,0.516129,85,217,197,...,-0.074037,0.016959,0.948476,0.008281,0.038995,0.890999,0.712799,0.015626,0.022831,0.000327
2208,0.447368,0.387833,0.373626,229,0.870722,233,0.823322,448,283,263,...,-0.037660,-0.007833,0.961154,-0.009217,0.006610,0.900462,0.840163,-0.021430,0.020557,0.004958


In [79]:
#choosing 80/20 split instead of 85/15 because of volume of data

dev, test = sklearn.model_selection.train_test_split(df, test_size=0.15, random_state=1)

train, val = sklearn.model_selection.train_test_split(dev, test_size=0.15, random_state=1)

print(dev.shape)
print(test.shape)
print(train.shape)
print(val.shape)

(1878, 12)
(332, 12)
(1596, 12)
(282, 12)


In [80]:
#ID target and features, separate into separate arrays

target = 'protein_match'
input_features = [columns for columns in df]
input_features.remove(target)

In [81]:
#split X and y

dev_X = dev[input_features].values
test_X = test[input_features].values

dev_y = dev[target].values.reshape(-1,1)
test_y = test[target].values.reshape(-1,1)  

print(dev_X.shape, test_X.shape, dev_y.shape, test_y.shape)

(1878, 11) (332, 11) (1878, 1) (332, 1)


In [82]:
#same thing for training and validation data

train_X = train[input_features].values
val_X = val[input_features].values

train_y = train[target].values.reshape(-1,1)
val_y = val[target].values.reshape(-1,1) 

Scale the data

In [83]:
scaler = sklearn.preprocessing.StandardScaler()
dev_X = scaler.fit_transform(dev_X)
test_X = scaler.fit_transform(test_X)
train_X = scaler.fit_transform(train_X)
val_X = scaler.fit_transform(val_X)

Train the model

In [84]:
#Random Forest

#hyperparameters determiend with optuna
model = sklearn.ensemble.RandomForestClassifier(n_estimators=150, max_depth=None, max_samples=0.5,
                                                max_features=0.5, min_weight_fraction_leaf=0.000215,
                                               min_samples_split=10)

model.fit(train_X, train_y.ravel())

RandomForestClassifier(max_features=0.5, max_samples=0.5, min_samples_split=10,
                       min_weight_fraction_leaf=0.000215, n_estimators=150)

Test the model, report relevant statistics

In [85]:
score = model.score(val_X, val_y)
print('Model score is: {}'.format(score))

preds = model.predict(test_X)
print(preds)

Model score is: 0.5212765957446809
['Yes' 'No' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No'
 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'Yes'
 'No' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes'
 'No' 'Yes' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No'
 'Yes' 'Yes' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No'
 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'No'
 'No' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No' 'Yes'
 'No' 'No' 'No' 'Yes' 'No' 'No' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes' 'No' 'No'
 'No' 'Yes' 'No' 'No' 'Yes' 'No' 'No' 'No' 'No' 'No' 'Yes' 'No' 'No' 'No'
 'No' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'No' 'No' 'No' 'No'
 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No'
 'Yes' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'Yes' 'No'
 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'No' 'No' 'Yes' 'Yes'
 'Yes' 'No

In [116]:
#confusion matrix

confusion_matrix = sklearn.metrics.confusion_matrix(preds, test_y)
sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix).plot()